# Time Series

The following notebook is for plotting a time series of the SECS dataset, as shown in Figure 8 in the paper

## Imports
A different backend is used to produe the figure in a new window as figures produced within notebook are limited in size and make it hard to see this type of figure

In [1]:
%matplotlib qt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import matplotlib as mpl
from apexpy import Apex
from matplotlib import dates as mdates

## Defintions required for having MLT and UT x axes

In [2]:
A=Apex()
def UT_to_mlt(UT):
    return [A.mlon2mlt(105, UT)]
@mpl.ticker.FuncFormatter
def MLTformatter(x, pos):
    """Assuming input x is a matplotlib datenum, convert x to MLT"""
    return "{:04.1f}".format(UT_to_mlt(mpl.dates.num2date(x))[0])

## Pick the start and end of the time series

In [3]:
start, end= np.datetime64('2000-01-28T01:00'), np.datetime64('2000-01-28T04:00')

## Loading the data and making selection

In [4]:
df= pd.read_csv('Meridian_Data.csv', index_col=0, parse_dates=['Date_UTC'])
df= df[(df.Date_UTC>=start)&(df.Date_UTC<=end)]
df.set_index('Date_UTC', inplace=True)
columns= df.columns
Cur_E= columns[1:51]
Cur_N= columns[51:101]
Br= columns[101:]

## Creating the figure

In [5]:
fig=plt.figure()
gs= fig.add_gridspec(4, 1, height_ratios= [.1, 1, .2, .05], hspace=0)
ax= fig.add_subplot(gs[1])
cax= fig.add_subplot(gs[3])
cbar=fig.colorbar(mpl.cm.ScalarMappable(norm = mpl.colors.Normalize(vmin=-2E2, vmax=2E2), 
                                        cmap=mpl.cm.seismic), cax=cax, 
                  orientation='horizontal', extend='both')

In [6]:
mlat=np.linspace(49, 81)
cf= ax.contourf(np.array([np.vstack(df.index.values)]*50).T[0],
                np.array([mlat]*len(df.index.values)), df[Br].values*1e9,
                cmap=mpl.cm.seismic, levels=100, vmin=-2e2, vmax=2e2)
cf.set_clim(-2e2, 2e2)

In [7]:
df=df.resample('5min').mean()
CurQ=ax.quiver(np.array([np.vstack(df.index.values)]*25).T.flatten(), 
          np.array([mlat[::2]]*len(df.index.values)).flatten(),
          df[Cur_E[::2]].values.flatten(), df[Cur_N[::2]].values.flatten(),
          scale=20, width=.003)

In [8]:
ax2 = ax.twiny()
_ = ax2.set_xticks(ax.get_xticks())
_ = ax2.set_xbound(ax.get_xbound())
ax2.xaxis.set_major_formatter(MLTformatter)
ax2.set_xlabel('MLT')
ax.set_xlabel('Universal Time')
ax.tick_params(axis='x', labelsize= 18, width=3) 
ax.tick_params(axis='y', labelsize= 18, width=3)
ax2.tick_params(axis='x', labelsize= 18, width=3) 
ax2.tick_params(axis='y', labelsize= 18, width=3)
ax.tick_params(axis='x', labelsize= 18, width=3) 
ax.tick_params(axis='y', labelsize= 18, width=3)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.quiverkey(CurQ, .5, -0.75, 500e-3, r'500$Akm^{-1}$')
ax.set_ylabel('Magnetic Latitude')
cbar.set_label(r'Br (nT)')
fig.suptitle(r'105$^\circ$ Meridian Time Series')

Text(0.5, 0.98, '105$^\\circ$ Meridian Time Series')